In [1]:
"""
Filter USDT Transactions from General Token Transfer File
Extracts only USDT transfers and creates whale vs retail analysis
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# ============================================================================
# CONFIGURATION
# ============================================================================
# Your general token transfer file (contains ALL tokens)
INPUT_FILE = '../data/token_transfers.csv'  # <-- CHANGE THIS TO YOUR FILE

# USDT Contract Address (Ethereum mainnet)
USDT_CONTRACT = '0xdac17f958d2ee523a2206206994597c13d831ec7'  # <-- Verify this matches your data

# Investor thresholds
WHALE_THRESHOLD = 100000      # > $100k
MEDIUM_THRESHOLD = 10000      # $10k - $100k
SMALL_THRESHOLD = 1000        # $1k - $10k

CRISIS_DATE = '2022-05-10'
# ============================================================================

print("="*70)
print("STEP 1: Loading and Filtering Data")
print("="*70)

# Load the full dataset
print(f"\nLoading all token transfers from: {INPUT_FILE}")
df_all = pd.read_csv(INPUT_FILE)

print(f"✓ Loaded {len(df_all):,} total token transfers")
print(f"✓ Columns: {list(df_all.columns)}")

# Check contract address column name (might be 'contract_address' or 'token_address')
contract_col = None
for col in df_all.columns:
    if 'contract' in col.lower() or 'token' in col.lower():
        if col != 'token_name':  # Skip token name column
            contract_col = col
            break

if contract_col is None:
    print("\n⚠ Warning: Could not auto-detect contract address column")
    print("Available columns:", df_all.columns.tolist())
    print("\nPlease update the script with the correct column name")
    exit()

print(f"\nUsing contract address column: '{contract_col}'")

# Convert contract addresses to lowercase for comparison
df_all[contract_col] = df_all[contract_col].str.lower()
USDT_CONTRACT = USDT_CONTRACT.lower()

# Filter for USDT only
print(f"\nFiltering for USDT contract: {USDT_CONTRACT}")
df_usdt = df_all[df_all[contract_col] == USDT_CONTRACT].copy()

print(f"✓ Found {len(df_usdt):,} USDT transactions ({len(df_usdt)/len(df_all)*100:.2f}% of total)")

if len(df_usdt) == 0:
    print("\n❌ No USDT transactions found!")
    print("\nAvailable contract addresses in your data:")
    print(df_all[contract_col].value_counts().head(10))
    print("\nPlease check and update USDT_CONTRACT in the script")
    exit()

# Save filtered USDT data (optional)
output_file = 'usdt_transfers_only.csv'
df_usdt.to_csv(output_file, index=False)
print(f"✓ Saved USDT-only data to: {output_file}")

print("\n" + "="*70)
print("STEP 2: Processing USDT Data")
print("="*70)

# Prepare datetime and value columns
df_usdt['datetime'] = pd.to_datetime(df_usdt['time_stamp'], unit='s')
df_usdt['value'] = pd.to_numeric(df_usdt['value'], errors='coerce')
df_usdt = df_usdt[df_usdt['value'] > 0].copy()

print(f"\n✓ Date range: {df_usdt['datetime'].min()} to {df_usdt['datetime'].max()}")
print(f"✓ Value range: ${df_usdt['value'].min():.2f} to ${df_usdt['value'].max():,.2f}")

# Categorize investors
def categorize_investor(value):
    if value >= WHALE_THRESHOLD:
        return 'Whale (>$100k)'
    elif value >= MEDIUM_THRESHOLD:
        return 'Medium ($10k-$100k)'
    elif value >= SMALL_THRESHOLD:
        return 'Small ($1k-$10k)'
    else:
        return 'Retail (<$1k)'

df_usdt['investor_type'] = df_usdt['value'].apply(categorize_investor)

print("\n" + "="*70)
print("STEP 3: Investor Distribution")
print("="*70)

print("\nTransaction Count by Investor Type:")
print(df_usdt['investor_type'].value_counts().sort_index())

print("\nTotal Value by Investor Type:")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    total = df_usdt[df_usdt['investor_type'] == inv_type]['value'].sum()
    pct = total / df_usdt['value'].sum() * 100
    print(f"  {inv_type:25} ${total:15,.2f} ({pct:5.1f}%)")

print("\n" + "="*70)
print("STEP 4: Creating Visualizations")
print("="*70)

# Aggregate data
hourly_value = df_usdt.groupby([df_usdt['datetime'].dt.floor('h'), 'investor_type'])['value'].sum().reset_index()
hourly_value.columns = ['datetime', 'investor_type', 'total_value']

hourly_count = df_usdt.groupby([df_usdt['datetime'].dt.floor('h'), 'investor_type']).size().reset_index(name='count')
hourly_count.columns = ['datetime', 'investor_type', 'count']

# Colors
colors = {
    'Whale (>$100k)': '#e74c3c',
    'Medium ($10k-$100k)': '#f39c12',
    'Small ($1k-$10k)': '#3498db',
    'Retail (<$1k)': '#2ecc71'
}

# ============================================================================
# VISUALIZATION 1: Transaction VALUE Timeline
# ============================================================================
print("\n📈 Creating: Transaction VALUE Timeline...")

fig, ax = plt.subplots(figsize=(18, 9))

for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_value[hourly_value['investor_type'] == inv_type]
    if len(data) > 0:
        ax.plot(data['datetime'], data['total_value'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

crisis_dt = pd.to_datetime(CRISIS_DATE)
ax.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7, 
           label='May 10, 2022 (UST Collapse)')

ax.set_title('USDT: Who Moved The Most Money? Transaction Value by Investor Type', 
             fontsize=20, fontweight='bold', pad=20)
ax.set_xlabel('Date/Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Total Transaction Value ($)', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper left', framealpha=0.9)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('usdt_whale_retail_VALUE.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: usdt_whale_retail_VALUE.png")
plt.close()

# ============================================================================
# VISUALIZATION 2: Transaction COUNT Timeline
# ============================================================================
print("📊 Creating: Transaction COUNT Timeline...")

fig, ax = plt.subplots(figsize=(18, 9))

for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_count[hourly_count['investor_type'] == inv_type]
    if len(data) > 0:
        ax.plot(data['datetime'], data['count'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

ax.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7, 
           label='May 10, 2022 (UST Collapse)')

ax.set_title('USDT: Who Panicked First? Transaction Count by Investor Type', 
             fontsize=20, fontweight='bold', pad=20)
ax.set_xlabel('Date/Time', fontsize=14, fontweight='bold')
ax.set_ylabel('Number of Transactions', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper left', framealpha=0.9)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('usdt_whale_retail_COUNT.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: usdt_whale_retail_COUNT.png")
plt.close()

# ============================================================================
# VISUALIZATION 3: Combined View
# ============================================================================
print("🎯 Creating: Combined Timeline (VALUE + COUNT)...")

fig, axes = plt.subplots(2, 1, figsize=(20, 12))

# Panel 1: VALUE
ax1 = axes[0]
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_value[hourly_value['investor_type'] == inv_type]
    if len(data) > 0:
        ax1.plot(data['datetime'], data['total_value'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

ax1.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7, label='May 10, 2022')
ax1.set_title('Transaction VALUE by Investor Type', fontsize=16, fontweight='bold')
ax1.set_ylabel('Total Transaction Value ($)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=11, loc='upper left', framealpha=0.9)
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')
ax1.tick_params(axis='x', labelbottom=False)

# Panel 2: COUNT
ax2 = axes[1]
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    data = hourly_count[hourly_count['investor_type'] == inv_type]
    if len(data) > 0:
        ax2.plot(data['datetime'], data['count'], 
                linewidth=3, label=inv_type, color=colors[inv_type], alpha=0.8)

ax2.axvline(crisis_dt, color='red', linestyle='--', linewidth=3, alpha=0.7)
ax2.set_title('Transaction COUNT by Investor Type', fontsize=16, fontweight='bold')
ax2.set_xlabel('Date/Time', fontsize=12, fontweight='bold')
ax2.set_ylabel('Number of Transactions', fontsize=12, fontweight='bold')
ax2.legend(fontsize=11, loc='upper left', framealpha=0.9)
ax2.grid(True, alpha=0.3)
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)

plt.suptitle('USDT Whale vs Retail: Complete Timeline Analysis', 
             fontsize=22, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('usdt_whale_retail_COMBINED.png', dpi=300, bbox_inches='tight')
print("  ✓ Saved: usdt_whale_retail_COMBINED.png")
plt.close()

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================
print("\n" + "="*70)
print("STEP 5: Before/After Crisis Analysis")
print("="*70)

before = df_usdt[df_usdt['datetime'] < crisis_dt]
after = df_usdt[df_usdt['datetime'] >= crisis_dt]

print("\nBEFORE Crisis (Before May 10, 2022):")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    count = len(before[before['investor_type'] == inv_type])
    total = before[before['investor_type'] == inv_type]['value'].sum()
    print(f"  {inv_type:25} Count: {count:6,} | Value: ${total:15,.2f}")

print("\nAFTER Crisis (After May 10, 2022):")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    count = len(after[after['investor_type'] == inv_type])
    total = after[after['investor_type'] == inv_type]['value'].sum()
    print(f"  {inv_type:25} Count: {count:6,} | Value: ${total:15,.2f}")

print("\nCHANGE (%):")
for inv_type in ['Whale (>$100k)', 'Medium ($10k-$100k)', 'Small ($1k-$10k)', 'Retail (<$1k)']:
    count_b = len(before[before['investor_type'] == inv_type])
    count_a = len(after[after['investor_type'] == inv_type])
    count_chg = ((count_a - count_b) / count_b * 100) if count_b > 0 else 0
    
    value_b = before[before['investor_type'] == inv_type]['value'].sum()
    value_a = after[after['investor_type'] == inv_type]['value'].sum()
    value_chg = ((value_a - value_b) / value_b * 100) if value_b > 0 else 0
    
    print(f"  {inv_type:25} Count: {count_chg:+7.1f}% | Value: {value_chg:+7.1f}%")

print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE!")
print("="*70)
print("\nGenerated Files:")
print("  1. usdt_transfers_only.csv           (Filtered USDT data)")
print("  2. usdt_whale_retail_VALUE.png       (Transaction value timeline)")
print("  3. usdt_whale_retail_COUNT.png       (Transaction count timeline)")
print("  4. usdt_whale_retail_COMBINED.png    (Both in one)")
print("="*70)

STEP 1: Loading and Filtering Data

Loading all token transfers from: ../data/token_transfers.csv
✓ Loaded 5,280,131 total token transfers
✓ Columns: ['block_number', 'transaction_index', 'from_address', 'to_address', 'time_stamp', 'contract_address', 'value']

Using contract address column: 'contract_address'

Filtering for USDT contract: 0xdac17f958d2ee523a2206206994597c13d831ec7
✓ Found 2,652,911 USDT transactions (50.24% of total)
✓ Saved USDT-only data to: usdt_transfers_only.csv

STEP 2: Processing USDT Data

✓ Date range: 2022-04-28 00:30:15 to 2022-05-25 18:47:44
✓ Value range: $0.00 to $3,000,000,000.00

STEP 3: Investor Distribution

Transaction Count by Investor Type:
investor_type
Medium ($10k-$100k)     423947
Retail (<$1k)          1163387
Small ($1k-$10k)        882860
Whale (>$100k)          181530
Name: count, dtype: int64

Total Value by Investor Type:
  Whale (>$100k)            $188,958,266,147.72 ( 91.6%)
  Medium ($10k-$100k)       $13,920,783,197.20 (  6.7%)
  Sm